In [1]:
import argparse
import os
import numpy as np
import math
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def read_data():
    root_dir = 'drive/My Drive/SAMPLE/faces94'
    files = glob(f"{root_dir}/**/**/*.jpg")

    c2i = {
        "female": 0,
        "male":1,
        "malestaff": 2 
    }
    return files,c2i

def display():
    print("for batch"+str(len(train_loader)))
    print(" -->> loss of the PartGen part and PartDicri part are :" +  str(g_loss.item()) + "  ," +  str(d_loss.item()))
    
def imgs_labels(c2i,files):
  Tlabel = torch.tensor([c2i[x.split('/')[-3]] for x in files])
  Timgs = torch.stack([TLoader(Image.open(x)) for x in files])
  return Tlabel,Timgs




In [0]:
epochs = 10
batch_size = 32
LRate = 0.001
IShape = (3, 200, 180)
interval = 100
bias1 = 0.5
bias2 = 0.999
hiddenUnits = 200

os.makedirs("images", exist_ok=True)
files,c2i=read_data()

In [0]:
class PartDiscri(nn.Module):
    def __init__(self):
        super(PartDiscri, self).__init__()
        #print("----",int(np.prod(IShape)))

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(IShape)), 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, img):
        
        img_flat = img.view(img.shape[0], -1)
        allowedity = self.model(img_flat)

        return allowedity
      
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(3*200*180,2000 ),nn.Tanh(),nn.Linear(2000, 1000),nn.Tanh(),nn.Linear(1000, 200),nn.Tanh(),nn.Linear(200, 100),)
        self.decoder = nn.Sequential(nn.Linear(100, 200),nn.Tanh(),nn.Linear(200, 1000),nn.Tanh(),nn.Linear(1000, 2000),nn.Tanh(),nn.Linear(2000, 3*200*180),nn.Sigmoid(),)
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded   
    

In [0]:
LOSS = torch.nn.BCELoss()


PartGen = AutoEncoder()
PartDiscri = PartDiscri()

OGEN = torch.optim.Adam(PartGen.parameters(), lr=LRate, betas=(bias1, bias2))
ODISCRI = torch.optim.Adam(PartDiscri.parameters(), lr=LRate, betas=(bias1, bias2))

In [0]:
Tensor = torch.FloatTensor

TLoader = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

Tlabel = torch.tensor([c2i[x.split('/')[-3]] for x in files])
Timgs = torch.stack([TLoader(Image.open(x)) for x in files])

In [0]:

train_idx, test_idx = train_test_split(range(len(Timgs)), test_size=0.2, random_state=102)


train_data = TensorDataset(Timgs[train_idx], Tlabel[train_idx])
test_data = TensorDataset(Timgs[test_idx], Tlabel[test_idx])

train_loader = DataLoader(train_data, sampler=RandomSampler(train_data), batch_size=32)
test_loader = DataLoader(test_data, sampler=SequentialSampler(test_data), batch_size=32)

for epoch in range(epochs):
    for i, (imgs,_) in enumerate(train_loader):
        allowed = Variable(Tensor(imgs.size(0), 1).fill_(1.0), requires_grad=False)
        NOTallowed = Variable(Tensor(imgs.size(0), 1).fill_(0.0), requires_grad=False)
        real_imgs = Variable(imgs.type(Tensor))
        OGEN.zero_grad()
        s1=32
        s2=108000
        z = Variable(Tensor(np.random.normal(0, 1, (s1, s2))))
        gen_imgs = PartGen(z)
        g_loss = LOSS(PartDiscri(gen_imgs), allowed)

        g_loss.backward()
        OGEN.step()
        ODISCRI.zero_grad()

        # Measure PartDiscri's ability to classify real from generated samples
        real_loss = LOSS(PartDiscri(real_imgs), allowed)
        NOTallowed_loss = LOSS(PartDiscri(gen_imgs.detach()), NOTallowed)
        d_loss = (real_loss + NOTallowed_loss) / 2

        d_loss.backward()
        ODISCRI.step()

        display()

        batches_done = epoch * len(train_loader) + i
        if batches_done % interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)